# FTP Client
Import the library to start creating the client

In [78]:
from ftplib import FTP

The ftplib.FTP() creates a new instance of the FTP class. 

When host and port are given, a connection to the host is made with the connect() method.

In [79]:
ftp = FTP()
ftp.connect('localhost', 2121)

'220 pyftpdlib based ftpd ready.'

In [80]:
ftp.login(user='lcarnevale', passwd='lcarnevale')

'230 Login successful.'

The getwelcome() returns the welcome message sent by the server in reply to the initial connection. This message may contain some helpful information for the user.

In [81]:
ftp.getwelcome()

'220 pyftpdlib based ftpd ready.'

## Working with Directory
Once connected, it is interesting to know where you are in the directory structure. The pwd() function on the ftplib.FTP object provides this data.

In [82]:
ftp.pwd()

'/'

The next basic task you will probably want to do is list the files in a directory. The ftplib.FTP.dir() command will list all the files in your current directory.

It does not just provide the filename though, it provides a string that contains the permissions, whether it is a directory, byte size, modification timestamp, owner, and group information. It is formatted just like the output from an ls command. Since the output is a string, you will have to parse out the information from it manually using split() or regular expressions.

In [83]:
files = []
ftp.dir(files.append)
for file in files:
    print(file)

-rw-rw-r--   1 lcarnevale lcarnevale   146534 Nov 18 18:33 image-uploaded.jpg
-rw-rw-r--   1 lcarnevale lcarnevale       22 Nov 18 17:34 sample.txt
-rw-rw-r--   1 lcarnevale lcarnevale       53 Nov 18 18:33 text-renamed.txt


Let's create a directory using ftplib.FTP.mkd() and pass it the name of the directory. It will return a string containing the name of the directory created.

In [84]:
ftp.mkd('new-folder')

'/new-folder'

To switch to a different directory, use ftplib.FTP.cwd()

In [85]:
ftp.cwd('new-folder')

'250 "/new-folder" is the current directory.'

Get back to '/' directory

In [86]:
ftp.cwd('..')

'250 "/" is the current directory.'

To remove a directory, just use rmd() on your FTP object. A directory must be empty to delete it.

In [87]:
ftp.rmd('new-folder')

'250 Directory removed.'

## Working with Files
For text files use storlines() and for binary use storbinary()

In [88]:
# For text or binary file, always use `rb`
with open('to-upload/text-to-upload.txt', 'rb') as text_file:
    ftp.storlines('STOR text-uploaded.txt', text_file)
with open('to-upload/image-to-upload.jpg', 'rb') as image_file:
    ftp.storbinary('STOR image-uploaded.jpg', image_file)

To check the size of a file on a remote FTP server, you can simply use the size() function as demonstrated in the following example.

Depending on whether you want to check a text file or a binary file, you want to tell the FTP server what type you want to use. Use sendcmd() and pass the type with either TYPE I for image/binary data or TYPE A for ASCII text.

The size() function will return the size of the file in bytes.

In [89]:
try:
    # "Image" or binary data
    ftp.sendcmd('TYPE I')
    print(ftp.size('image-uploaded.jpg'))
except all_errors as error:
    print(f"Error checking image size: {error}")

try:
    # "ASCII" text
    ftp.sendcmd('TYPE I')  
    print(ftp.size('text-uploaded.txt'))
except all_errors as error:
    print(f"Error checking text file size: {error}")

146534
53


To rename a file on a remote FTP server, use the rename() function and pass the original filename and the new filename.

In [90]:
try:
    ftp.rename('text-uploaded.txt', 'text-renamed.txt')
except all_errors as error:
    print(f'Error renaming file on server: {error}')

In [91]:
# For text files
with open('to-upload/text-downloaded.txt', 'w') as local_file:
    response = ftp.retrlines('RETR text-renamed.txt', local_file.write)
    # Check the response code
    # https://en.wikipedia.org/wiki/List_of_FTP_server_return_codes
    if response.startswith('226'):  # Transfer complete
        print('Transfer complete')
    else:
        print('Error transferring. Local file may be incomplete or corrupt.')

# For binary use `retrbinary()`
with open('to-upload/image-downloaded.jpg', 'wb') as local_file:
    response = ftp.retrbinary('RETR image-uploaded.jpg', local_file.write)
    if response.startswith('226'):  # Transfer complete
        print('Transfer complete')
    else:
        print('Error transferring. Local file may be incomplete or corrupt.')

Transfer complete
Transfer complete


To delete a remote file, use the delete() function and give it the filename you want to delete. You cannot delete directories with this. For directories, you must use rmd() as shown in the example earlier.

In [92]:
try:
    ftp.delete('text-renamed.txt')
except all_errors as error:
    print(f'Error deleting file: {error}') 